<a href="https://colab.research.google.com/github/Levelleor/googlecollab/blob/main/colab_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Define the URL and the local name you want to give the model
HG_MODEL_URL = "https://huggingface.co/TeeZee/DarkForest-20B-v2.0-GGUF/resolve/main/DarkForest-20B-v2.0.q4_K_M.gguf"
LOCAL_FILENAME = "darkforest.gguf"
MODEL_NAME = "dark-forest"  # This is the name we will call in the API later

# Environment variables (Same as before)
%env OLLAMA_CONTEXT_LENGTH=16384
%env OLLAMA_HOST=0.0.0.0
%env OLLAMA_KEEP_ALIVE=-1



env: OLLAMA_CONTEXT_LENGTH=16384
env: OLLAMA_HOST=0.0.0.0
env: OLLAMA_KEEP_ALIVE=-1


In [2]:
!apt-get install -y lshw pciutils
!nvcc --version
!nvidia-smi

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print(f"\n🧠 Available RAM: {ram_gb:.1f} GB")
print("✅ High-RAM runtime!" if ram_gb >= 20 else "❌ Not a high-RAM runtime.")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids usb.ids
The following NEW packages will be installed:
  libpci3 lshw pci.ids pciutils usb.ids
0 upgraded, 5 newly installed, 0 to remove and 41 not upgraded.
Need to get 883 kB of archives.
After this operation, 3,256 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 883 kB in 2s (418 kB/s)
Selecting previously unselected package pc

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tgz
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [15]:
import subprocess
import time
import requests
import threading

# Start ollama serve in a background thread
def start_ollama():
    subprocess.call(['ollama', 'serve'])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.daemon = True
ollama_thread.start()

# A. Download the specific GGUF file
print(f"⬇️ Downloading {LOCAL_FILENAME}...")
!wget {HG_MODEL_URL} -O {LOCAL_FILENAME}

# B. Create a Modelfile
# We write a file named 'Modelfile' that tells Ollama where the weights are
with open("Modelfile", "w") as f:
    f.write(f"FROM ./{LOCAL_FILENAME}\n")
    # Optional: You can set a template here if the model requires a specific format (e.g., ChatML, Alpaca)
    # f.write("TEMPLATE \"{{ .System }}\nUser: {{ .Prompt }}\nAssistant: \"")

# C. Create the model inside Ollama
print("⚙️ Creating the model in Ollama...")
!ollama create {MODEL_NAME} -f Modelfile

# D. Verify it exists
!ollama list

# Wait for Ollama HTTP API to be ready
def wait_for_ollama(timeout=60):
    for i in range(timeout):
        try:
            r = requests.get("http://localhost:11434")
            if r.status_code in [200, 404]:
                print(f"✅ Ollama is up (after {i+1}s).")
                return
        except requests.exceptions.ConnectionError:
            pass
        print(f"⏳ Waiting for Ollama to start... {i+1}s")
        time.sleep(1)
    raise RuntimeError("❌ Ollama did not start in time.")

wait_for_ollama()


⬇️ Downloading darkforest.gguf...
--2026-01-14 04:11:45--  https://huggingface.co/TeeZee/DarkForest-20B-v2.0-GGUF/resolve/main/DarkForest-20B-v2.0.q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.97, 13.35.202.34, 13.35.202.121, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.97|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/65d24469dd8292fdc667b127/f89afdf9cf128212bb255608b9320346ba8b3a2f57d721dc441e0313f36b128f?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27DarkForest-20B-v2.0.q4_K_M.gguf%3B+filename%3D%22DarkForest-20B-v2.0.q4_K_M.gguf%22%3B&Expires=1768367505&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY4MzY3NTA1fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjVkMjQ0NjlkZDgyOTJmZGM2NjdiMTI3L2Y4OWFmZGY5Y2YxMjgyMTJiYjI1NTYwOGI5MzIwMzQ2YmE4YjNhMmY1N2Q3MjFkYzQ0MWUwMzEzZjM2YjEyOGZcXD9yZXNwb25zZS1jb250ZW50LWR

In [5]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared


--2026-01-14 03:47:03--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2026-01-14 03:47:03--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-14T04%3A38%3A45Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2026-0

In [12]:
import re

# Run cloudflared tunnel in background and get the public URL
cloudflared_proc = subprocess.Popen(
    ['./cloudflared', 'tunnel', '--url', 'http://localhost:11434', '--no-autoupdate'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

public_url = None
for line in cloudflared_proc.stdout:
    print(line.strip())
    match = re.search(r'(https://.*\.trycloudflare\.com)', line)
    if match:
        public_url = match.group(1)
        break

if public_url:
    print(f"\n✅ Public URL for Ollama:\n{public_url}")
else:
    raise RuntimeError("❌ Could not find public Cloudflare URL.")


2026-01-14T04:09:30Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2026-01-14T04:09:30Z INF Requesting new quick Tunnel on trycloudflare.com...
2026-01-14T04:09:34Z INF +--------------------------------------------------------------------------------------------+
2026-01-14T04:09:34Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2026-01-14T04:09:34Z INF |  https://margaret-precision-gotta-abstract.trycloudfla

In [11]:
import requests
import json

url = "http://localhost:11434/api/generate"

data = {
    "model": "dark-forest",
    "prompt": "Question: Write a short poem about a cyberpunk forest.\nAnswer:",
    "stream": True  # <--- ENABLE STREAMING
}

print(f"⏳ Sending request to {url}...")

try:
    # We use stream=True in the requests call to keep the connection open
    response = requests.post(url, json=data, stream=True)

    if response.status_code == 200:
        print("✅ Connection established! Generating:\n")
        # Iterate over the response line by line
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode('utf-8')
                json_part = json.loads(decoded_line)
                # Print the token immediately without a newline
                print(json_part.get("response", ""), end="", flush=True)

                if json_part.get("done"):
                    print("\n\n✅ Done!")
    else:
        print(f"❌ Error! Status Code: {response.status_code}")
        print(response.text)

except Exception as e:
    print(f"❌ Error: {e}")

⏳ Sending request to http://localhost:11434/api/generate...
✅ Connection established! Generating:

 In the heart of neon city's grind,
A forest stands defiant, out of time,
Trees bathed in holographic moonlight glow

KeyboardInterrupt: 

In [8]:
import json

data = {
    "model": MODEL_NAME,
    "prompt": "Question: What is the capital of Japan?\nAnswer:",
    "stream": False
}

response = requests.post(f"{public_url}/api/generate", json=data)
print(response.json())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [14]:
!ollama ps

Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it
